In [1]:
import torch
import os
import struct
import matplotlib.pyplot as plt
import numpy as np

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import copy

def read_all_images(path_to_data):
   """
   :param path_to_data: the file containing the binary images from the STL-10 dataset
   :return: an array containing all the images
   """

   with open(path_to_data, 'rb') as f:
       # read whole file in uint8 chunks
       everything = np.fromfile(f, dtype=np.uint8)

       images = np.reshape(everything, (-1, 3, 96, 96))
       

       images = np.transpose(images, (0, 3, 2, 1))
       images =  np.dot(images[:,:,:,0:3],[0.299,0.587,0.114])
       print(images.shape)
       images = np.reshape(images, (len(images), 96*96))
       return images
images = read_all_images(os.getcwd()+"/data/stl10_binary/train_X.bin")


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(96*96, 96*48),
            nn.ReLU(True),
            nn.Linear(96*48, 48*48),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(48*48,48*96),
            nn.ReLU(True),
            nn.Linear(48*96,96*96))
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


def l2_penalty(model2):
#    reg_lambda=1.0
   l2_r=0
   for weight in model2.encoder.parameters():
       for wi in weight:
           l2_r+= wi.norm(2)
       print("one layers parameters")
   return l2_r**0.5

def l1_penalty(model2):
    l1_r=0
    for weight in model2.parameters():
        l1_r += weight.abs().sum()
        print("one layers parameters")
            #print (l1_r)
    return l1_r
    
def Model_Caller(norm):
    model2 = autoencoder().cuda()
    criterion = nn.MSELoss()
    num_epochs = 100
    batch_size = 100
    learning_rate = 0.0001
    optimizer = optim.Adam(model2.parameters(), lr = learning_rate)
    for epoch in range(num_epochs):
        for i in range(0,5000, batch_size):
            batch_x = images[i:i+500]
            batch_x = Variable(torch.cuda.FloatTensor(batch_x))
            if norm ==1:
                l1_loss = l1_penalty(model2)
                norm_loss = l1_loss
            elif norm == 2:
                l2_loss = l2_penalty(model2)
                norm_loss = l2_loss
            else:
                l1_loss = l1_penalty(model2)
                l2_loss = l2_penalty(model2)
                elastic = 0.5*l1_loss +0.5*l2_loss
                norm_loss = elastic
            output = model2(batch_x)
    #         l2_loss = l2_penalty(model)

    #         elastic = 0.5*l1_loss +0.5*l2_loss
            loss = criterion(output, batch_x)
            loss+= norm_loss
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # ===================log========================
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data[0]))
    feature_rep = model2.encoder(images)
    return feature_rep
     
feature_rep = Model_Caller(1)
def Neural_nework(feature_rep):
    model_nn = NN(48*48,48*24,24*24,10).cuda()
    criterion = nn.MSELoss()
    learning_rate=0.001
    optimizer = torch.optim.SGD(model_nn.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        batch_x =feature_rep
        output = NeuralNet(batch_x)
        output = output.data.numpy()
        output = Variable(torch.cuda.FloatTensor(output),requires_grad=True)
        label_batch = label_oh
        label_batch = Variable(torch.cuda.FloatTensor(label_batch))
        loss = criterion(output,label_batch)
    #     print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 
        print (loss.data[0], "  in epoch  ", epoch)
        test_images = read_all_images(os.getcwd()+"/data/stl10_binary/test_X.bin")
        test_fea= model.encoder(Variable(torch.FloatTensor(test_images)))
        # test_fea=model_inner.encoder(test_fea)
        y_pred= NeuralNet(test_fea)
        y_pred = y_pred.cpu().detach().numpy()
        y_pred = y_pred+1
        y_pred = np.argmax(y_pred, axis=1)
        aa=Accuracy(y_pred,real)
        acc = aa/len(y_pred)
        print ("Accuracy :",acc)
        return acc
acc = Neural_nework(feature_rep)

    



(5000, 96, 96)


RuntimeError: CUDA error: out of memory